<a href="https://colab.research.google.com/github/anvichip/text-analysis-nlp/blob/main/web_scrape_text_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install beautifulsoup4
!pip install nltk
!pip install chardet
!pip install syllables

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.3/939.3 kB 20.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import os
import syllables
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import chardet
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
df = pd.read_excel('Input.xlsx')

In [4]:
df

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...
...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...
110,147,https://insights.blackcoffer.com/the-future-of...
111,148,https://insights.blackcoffer.com/big-data-anal...
112,149,https://insights.blackcoffer.com/business-anal...


In [5]:
def extract_text(link):
  text = ""
  p_texts = []
  # Send a GET request to the website
  try:
    response = requests.get(link)
    # Get the HTML content from the response
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    # Find the first occurrence of the <h1> element with the specified class
    h1_element = soup.find('h1')
    if h1_element is not None:
      # Extract the text content of the element
      h1_text = h1_element.text
      # Print the extracted text
      #print(h1_text)
    else:
      h1_text = ""
    #div_tag = soup.select('div.td-post-content.tagdiv-type','div.tdb-block-inner.td-fix-index')
    div_tag = soup.select('.td-post-content.tagdiv-type,.tdb-block-inner.td-fix-index')
    if div_tag is not None:
      for p in div_tag:
        p_tags = p.find_all('p')
        #print(p_tags)
        if len(p_tags) != 0:
          p_texts = [p.text for p in p_tags]
      #print(p_texts)
  #div_tag = soup.select('div.td-post-content tagdiv-type.tdb-block-inner td-fix-index')
  #print(div_tag.get('class'))
  # Find all the inner tags within the <div> tag and extract their text content
#   inner_tags_text = [tag.text for tag in div_tag.find_all('p')]
#   # Print the extracted text content
#  #print(inner_tags_text)
#   for para in inner_tags_text:
#     #print(text)
#     text = text + para + " "
  #print(h1_text)
  #print(p_texts)
    for i in p_texts:
      text = text + i + " "
    return h1_text,text
  except requests.exceptions.HTTPError as err:
    if err.response.status_code == 404:
      print("Error 404: Page not found")


In [6]:
# Specify the directory path where you want to create the folder
directory = '/content/text_files/'

# Create the folder using the mkdir command
!mkdir "{directory}"


## **Extract text of the URL's and create text files in the folder**

In [8]:
for index, rows in df.iterrows():
  name = rows['URL_ID']
  url = str(rows['URL'])
  #print(name)
  head, text = extract_text(url)
  # # head,text = extract_text(url)
  if head != "" and text != "":
    file_name = f'{name}.txt'
    file_path = os.path.join(directory, file_name)
    with open(file_path, 'w') as f:
      # Write the column values to the text file
      f.write(f'{head}\n')
      f.write(f'{text}\n')


## **Checking number of files and null files**

In [9]:
def count_files(folder_path):
    file_count = 0
    nul_count = 0
    only_head = 0
    for _, _, files in os.walk(folder_path):
        file_count += len(files)
    for txt in os.listdir(folder_path):
      if txt.endswith('.txt'):
        file_path = os.path.join(folder_path, txt)

        # Detect the encoding of the file
        with open(file_path, 'rb') as file:
            encoding = chardet.detect(file.read())['encoding']

        # Read the stop words from the file and add them to the set
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()
            words = content.split()
            num_words = len(words)
            if num_words < 20:
              only_head+=1
              print(file_path)
            if content == " ":
              nul_count+=1
    return file_count,nul_count,only_head

# Example usage
folder_path = '/content/text_files'  # Replace with the actual folder path
num_files,nul,head = count_files(folder_path)
print("Number of files:", num_files)
print("Number of null files:", nul)
print("Number of files only w heading", head)

Number of files: 111
Number of null files: 0
Number of files only w heading 0


In [85]:
print(df.loc[df['URL_ID'] == 144])
print(df.loc[df['URL_ID'] == 57])
print(df.loc[df['URL_ID'] == 44])

     URL_ID                                                URL
107     144  https://insights.blackcoffer.com/ensuring-grow...
    URL_ID                                                URL
20      57  https://insights.blackcoffer.com/covid-19-envi...
   URL_ID                                                URL
7      44  https://insights.blackcoffer.com/how-neural-ne...


In [49]:
# folder_path = '/content/text_files/106.txt'

# only_head = 0
# nul_count = 0
# file_count = 0
# # Detect the encoding of the file
# with open(folder_path, 'rb') as file:
#     encoding = chardet.detect(file.read())['encoding']

# # Read the stop words from the file and add them to the set
# with open(folder_path, 'r', encoding=encoding) as file:
#     content = file.read()
#     words = content.split()
#     num_words = len(words)
#     print(type(num_words))
#     print(content)
#     print(words)
#     print(num_words)
#     if num_words < 15:
#       only_head+=1
#     if content == " ":
#       nul_count+=1
# print(file_count,nul_count,only_head)


<class 'int'>
Contribution of handicrafts (Visual Arts & Literature) in the Indian economy
Handicrafts is an art of making crafts by hand in India is called Indian handicrafts, in ancient India people lived in colonies called tribals and they were used to make utility items for their daily needs, that art of making the crafts called handicrafts, and the items called handicrafts goods. Few of the Indian handicrafts are as follows The visual arts are art forms such as painting, drawing, printmaking, sculpture, ceramics, photography, video, filmmaking, design, crafts, and architecture. Many artistic disciplines such as performing arts, conceptual art, textile arts also involve aspects of visual arts as well as arts of other types. Also included within the visual arts are the applied arts such as industrial design, graphic design, fashion design, interior design, and decorative art. Roles of handicrafts in Indian economy Some key facts of Indian handicrafts industry 

['Contribution', 'of'

## **Removing Stopwords**

In [10]:
!unzip /content/StopWords-20230618T144933Z-001.zip

Archive:  /content/StopWords-20230618T144933Z-001.zip
  inflating: StopWords/StopWords_Currencies.txt  
  inflating: StopWords/StopWords_Auditor.txt  
  inflating: StopWords/StopWords_DatesandNumbers.txt  
  inflating: StopWords/StopWords_Names.txt  
  inflating: StopWords/StopWords_Generic.txt  
  inflating: StopWords/StopWords_Geographic.txt  
  inflating: StopWords/StopWords_GenericLong.txt  


In [11]:
# Specify the directory path where the text files are located
directory = '/content/StopWords/'

# Create an empty set to store all stop words
stop_words = set()

# Loop through the files in the stop words folder
stop_words_folder = '/content/StopWords/'
for filename in os.listdir(stop_words_folder):
    if filename.endswith('.txt'):
        file_path = os.path.join(stop_words_folder, filename)

        # Detect the encoding of the file
        with open(file_path, 'rb') as file:
            encoding = chardet.detect(file.read())['encoding']

        # Read the stop words from the file and add them to the set
        with open(file_path, 'r', encoding=encoding) as file:
            stop_words.update(word.strip() for word in file)

# Loop through the text files
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory, filename)

        # Detect the encoding of the file
        with open(file_path, 'rb') as file:
            encoding = chardet.detect(file.read())['encoding']

        # Read the file content
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()

        # Tokenize the content into words
        words = word_tokenize(content)

        # Remove stop words
        filtered_words = [word for word in words if word.lower() not in stop_words]

        # Join the filtered words back into a string
        filtered_content = ' '.join(filtered_words)

        # Write the filtered content back to the file
        with open(file_path, 'w', encoding=encoding) as file:
            file.write(filtered_content)


In [12]:
!unzip /content/MasterDictionary-20230618T144932Z-001.zip

Archive:  /content/MasterDictionary-20230618T144932Z-001.zip
  inflating: MasterDictionary/positive-words.txt  
  inflating: MasterDictionary/negative-words.txt  


In [13]:
positive_file = '/content/MasterDictionary/positive-words.txt'
negative_file = '/content/MasterDictionary/negative-words.txt'
positive_words = set()
negative_words = set()
with open(positive_file, 'rb') as file:
  encoding = chardet.detect(file.read())['encoding']
with open(positive_file, 'r',encoding = encoding) as file:
    positive_words.update(word.strip() for word in file if word.strip() not in stop_words)
with open(negative_file, 'rb') as file:
  encoding = chardet.detect(file.read())['encoding']
with open(negative_file, 'r',encoding = encoding) as file:
    negative_words.update(word.strip() for word in file if word.strip() not in stop_words)

In [14]:
print(positive_words)
print(negative_words)

{'assuredly', 'spectacular', 'dynamic', 'realizable', 'enhancement', 'headway', 'overtook', 'divine', 'won', 'thrillingly', 'acclaim', 'respect', 'tremendously', 'wisely', 'ecenomical', 'noble', 'wows', 'diligent', 'hard-working', 'issue-free', 'fervidly', 'first-class', 'ecstasy', 'gumption', 'glow', 'meticulously', 'responsive', 'unparalleled', 'refreshed', 'non-violent', 'simplest', 'success', 'tenaciously', 'noiseless', 'recommend', 'interesting', 'well-behaved', 'benevolent', 'inestimable', 'ecstatically', 'ebullient', 'divinely', 'righteous', 'tingle', 'greatness', 'spellbinding', 'reliable', 'enthrall', 'exceeded', 'delicious', 'conciliatory', 'reverent', 'patriot', 'fondness', 'protective', 'cure-all', 'resplendent', 'promoter', 'favorited', 'enthral', 'godsend', 'infallible', 'enhances', 'hearten', 'innovative', 'prosperous', 'diligence', 'intuitive', 'dependably', 'contentment', 'principled', 'luckiness', 'worth', 'sharp', 'dignify', 'richer', 'expansive', 'wonderfully', 'fre

In [15]:
# Loop through the text files
directory = '/content/text_files'
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory, filename)

        # Detect the encoding of the file
        with open(file_path, 'rb') as file:
            encoding = chardet.detect(file.read())['encoding']

        # Read the file content
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()

        # Tokenize the content into words
        words = word_tokenize(content)

        # Remove stop words
        filtered_words = [word for word in words if word.lower() not in stop_words]

        # Join the filtered words back into a string
        filtered_content = ' '.join(filtered_words)

        # Write the filtered content back to the file
        with open(file_path, 'w', encoding=encoding) as file:
            file.write(filtered_content)


## **1. Extracting Derived variables**


In [16]:
def calculate_polarity_subjectivity(pos,neg,total):
  polarity = (pos - neg)/ ((pos + neg) + 0.000001)
  subject =  (pos + neg)/ ((total) + 0.000001)
  return polarity,subject


def score(text):
    """Returns a score btw -1 and 1"""
    text = [e.lower() for e in text if e.isalnum()]
    #total = len(text)
    pos = len([e for e in text if e in positive_words])
    neg = len([e for e in text if e in negative_words])
    return pos, neg


## **2. Analysis of Readability & 4. Complex Words**

In [17]:
import re
def count_syllables(word):
    word = word.lower()
    if word.endswith(('es', 'ed')):
        word = word[:-2]
    syllables = re.findall(r'[aeiouy]+', word)
    return len(syllables)

def calculate_readability(text):
    # Tokenize the text into words
    words = nltk.word_tokenize(text)

    # Count the number of words and sentences
    num_words = len(words)
    num_sentences = len(nltk.sent_tokenize(text))
    num_syllables = sum(count_syllables(word) for word in words)
    if num_sentences > 0:
        # Calculate the average sentence length
        avg_sentence_length = num_words / num_sentences

        # Count the number of complex words (words with more than 2 syllables)
        complex_words = [word for word in words if len(word) > 2 and syllables.estimate(word) > 2]
        num_complex_words = len(complex_words)

        # Calculate the percentage of complex words
        percentage_complex_words = (num_complex_words / num_words) * 100

        # Calculate the Fog Index
        fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)

        return avg_sentence_length, percentage_complex_words, fog_index,num_complex_words
    else:
        # Return default values or handle the case when there are no sentences
        return 0, 0, 0, 0


## **3. Average Number of Words Per Sentence**


In [18]:
def calculate_average_words_per_sentence(file_path):
    with open(file_path, 'rb') as file:
        encoding = chardet.detect(file.read())['encoding']
    with open(file_path, 'r',encoding = encoding) as file:
        text = file.read()

    sentences = nltk.sent_tokenize(text)
    num_sentences = len(sentences)

    words = nltk.word_tokenize(text)
    num_words = len(words)
    if num_sentences > 0:
      average_words_per_sentence = num_words / num_sentences
      return average_words_per_sentence
    else:
      return 0


## **5. WORD COUNT**

In [19]:
import os
import string
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def count_cleaned_words(file_path):
  with open(file_path, 'rb') as file:
    encoding = chardet.detect(file.read())['encoding']

  with open(file_path, 'r', encoding = encoding) as file:
      text = file.read()

  # Remove punctuation
  text = text.translate(str.maketrans('', '', string.punctuation))

  # Tokenize the text into words
  words = text.split()

  # Remove stop words
  cleaned_words = [word for word in words if word.lower() not in stop_words]

  # Count the cleaned words
  num_cleaned_words = len(cleaned_words)

  return num_cleaned_words

# Path to the folder containing the text files
# text_files_folder = '/content/text_files'

# # Loop through all files in the folder
# for filename in os.listdir(text_files_folder):
#     if filename.endswith('.txt'):
#         file_path = os.path.join(text_files_folder, filename)
#         num_cleaned_words = count_cleaned_words(file_path)
#         print(f'File: {filename}')
#         print(f'Total Cleaned Words: {num_cleaned_words}')
#         print('---')


## **6. Syllable Count Per Word**


In [20]:
def count_syllables(word):
    word = word.lower()
    if word.endswith(('es', 'ed')):
        word = word[:-2]
    syllables = re.findall(r'[aeiouy]+', word)
    return len(syllables)

def syllable(text):
  words = nltk.word_tokenize(text)
  # Count the number of words and sentences
  #num_words = len(words)
  #num_sentences = len(nltk.sent_tokenize(text))
  num_syllables = sum(count_syllables(word) for word in words)
  return num_syllables


## **7. Personal Pronouns**

In [21]:
import os
import re

def count_personal_pronouns(file_path):
  with open(file_path, 'rb') as file:
    encoding = chardet.detect(file.read())['encoding']
  with open(file_path, 'r',encoding = encoding) as file:
    text = file.read()

  # Define the personal pronouns regex pattern
  pattern = r'\b(I|we|my|ours|us)\b'

  # Find all matches of personal pronouns in the text
  matches = re.findall(pattern, text, flags=re.IGNORECASE)

  # Count the personal pronouns
  num_personal_pronouns = len(matches)

  return num_personal_pronouns

# # Path to the folder containing the text files
# text_files_folder = '/content/text_files'

# # Loop through all files in the folder
# for filename in os.listdir(text_files_folder):
#     if filename.endswith('.txt'):
#         file_path = os.path.join(text_files_folder, filename)
#         num_personal_pronouns = count_personal_pronouns(file_path)
#         print(f'File: {filename}')
#         print(f'Personal Pronouns Count: {num_personal_pronouns}')
#         print('---')


## **8. Average Word length**

In [22]:
def calculate_average_word_length(file_path):

    # Detect the encoding of the file
    with open(file_path, 'rb') as file:
        encoding = chardet.detect(file.read())['encoding']
    with open(file_path, 'r',encoding = encoding) as file:
        text = file.read()

    # Split the text into words
    words = text.split()

    # Calculate the total number of characters in each word
    total_characters = sum(len(word) for word in words)

    # Calculate the total number of words
    num_words = len(words)
    if num_words > 0:
      # Calculate the average word length
      average_word_length = total_characters / num_words
      return average_word_length
    else:
      return 0

# Path to the folder containing the text files
# text_files_folder = '/content/text_files'

# # Loop through all files in the folder
# for filename in os.listdir(text_files_folder):
#     if filename.endswith('.txt'):
#         file_path = os.path.join(text_files_folder, filename)
#         average_word_length = calculate_average_word_length(file_path)
#         print(f'File: {filename}')
#         print(f'Average Word Length: {average_word_length:.2f}')
#         print('---')


In [47]:
data = []
for filename in os.listdir(directory):
      if filename.endswith('.txt'):
          file_path = os.path.join(directory, filename)
          with open(file_path, 'rb') as file:
            encoding = chardet.detect(file.read())['encoding']
          with open(file_path, 'r', encoding=encoding) as file:
              text = file.read()
              tokens = word_tokenize(text)
              pos,neg = score(tokens)
              total = len(text)
              # positive_count = sum(1 for token in tokens if token.lower() in positive_words)
              # negative_count = sum(1 for token in tokens if token.lower() in negative_words)
              polarity, subjectivity = calculate_polarity_subjectivity(pos,neg,total)
              avg_sentence_length,percentage_complex_words,fog_index,complex_words = calculate_readability(text)
              avg_word_per_sentence = calculate_average_words_per_sentence(file_path)
              average_word_length = calculate_average_word_length(file_path)
              word_count = count_cleaned_words(file_path)
              num_syllable = syllable(text)
              personal_pronouns = count_personal_pronouns(file_path)
              new_file = filename.replace('.txt', "")
              avg_word_len = calculate_average_word_length(file_path)
              print(f'File: {filename}')
              print(f'Positive Count: {pos}')
              print(f'Negative Count: {neg}')
              print(f'Polarity: {polarity}')
              print(f'Subjectivity: {subjectivity}')
              print(f'Avg Sentence Length: {avg_sentence_length}')
              print(f'Percentage Complex Words: {percentage_complex_words}')
              print(f'Fog Index: {fog_index}')
              print(f'Avg Words Per Sentence: {avg_word_per_sentence}')
              print(f'Average Word Length: {average_word_length:.2f}')
              print(f'Complex Word Count: {complex_words}')
              print(f'Word Count: {word_count}')
              print(f'Syllables per word: {num_syllable}')
              print(f'Personal Pronouns: {word_count}')
              print(f'Average Word Length: {average_word_length:.2f}')
              print('---')

              data.append({
                'URL_ID': new_file,
                #'AVERAGE NUMBER OF WORDS PER SENTENCE': avg_word_per_sentence,
                'POSITIVE SCORE': pos,
                'NEGATIVE SCORE': neg,
                'POLARITY SCORE': polarity,
                'SUBJECTIVITY SCORE':subjectivity,
                'AVG SENTENCE LENGTH': avg_sentence_length,
                'PERCENTAGE OF COMPLEX WORDS': percentage_complex_words,
                'FOG INDEX': fog_index,
                'AVG NUMBER OF WORDS PER SENTENCE':avg_word_per_sentence,
                #'':average_word_length,
                'COMPLEX WORD COUNT': complex_words,
                'WORD COUNT':word_count,
                'SYLLABLE PER WORD': num_syllable,
                'PERSONAL PRONOUNS': word_count,
                'AVG WORD LENGTH':average_word_length
        })
df_new = pd.DataFrame(data)

File: 84.txt
Positive Count: 20
Negative Count: 12
Polarity: 0.24999999218750027
Subjectivity: 0.012514665618883588
Avg Sentence Length: 13.1
Percentage Complex Words: 33.078880407124686
Fog Index: 18.471552162849875
Avg Words Per Sentence: 13.1
Average Word Length: 5.53
Complex Word Count: 130
Word Count: 323
Syllables per word: 682
Personal Pronouns: 323
Average Word Length: 5.53
---
File: 81.txt
Positive Count: 22
Negative Count: 35
Polarity: -0.2280701714373654
Subjectivity: 0.014372163385180999
Avg Sentence Length: 14.44186046511628
Percentage Complex Words: 24.315619967793882
Fog Index: 15.502992173164067
Avg Words Per Sentence: 14.44186046511628
Average Word Length: 5.39
Complex Word Count: 151
Word Count: 525
Syllables per word: 968
Personal Pronouns: 525
Average Word Length: 5.39
---
File: 59.txt
Positive Count: 17
Negative Count: 14
Polarity: 0.09677419042663901
Subjectivity: 0.010649261418533404
Avg Sentence Length: 14.89655172413793
Percentage Complex Words: 33.101851851851

In [48]:
df_new = pd.DataFrame(data)

In [49]:
print(df_new)

    URL_ID  POSITIVE SCORE  NEGATIVE SCORE  POLARITY SCORE  \
0       84              20              12        0.250000   
1       81              22              35       -0.228070   
2       59              17              14        0.096774   
3       52              26               0        1.000000   
4      139               8               3        0.454545   
..     ...             ...             ...             ...   
106    114              48              48        0.000000   
107    129               5               7       -0.166667   
108     91               3               1        0.500000   
109     74              34              25        0.152542   
110    133              26              20        0.130435   

     SUBJECTIVITY SCORE  AVG SENTENCE LENGTH  PERCENTAGE OF COMPLEX WORDS  \
0              0.012515            13.100000                    33.078880   
1              0.014372            14.441860                    24.315620   
2              0.010649 

In [56]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   URL_ID                            111 non-null    int64  
 1   POSITIVE SCORE                    111 non-null    int64  
 2   NEGATIVE SCORE                    111 non-null    int64  
 3   POLARITY SCORE                    111 non-null    float64
 4   SUBJECTIVITY SCORE                111 non-null    float64
 5   AVG SENTENCE LENGTH               111 non-null    float64
 6   PERCENTAGE OF COMPLEX WORDS       111 non-null    float64
 7   FOG INDEX                         111 non-null    float64
 8   AVG NUMBER OF WORDS PER SENTENCE  111 non-null    float64
 9   COMPLEX WORD COUNT                111 non-null    int64  
 10  WORD COUNT                        111 non-null    int64  
 11  SYLLABLE PER WORD                 111 non-null    int64  
 12  PERSONAL

In [59]:
excel_file = pd.read_excel('/content/Output Data Structure.xlsx')
excel_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 15 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   URL_ID                            114 non-null    int64  
 1   URL                               114 non-null    object 
 2   POSITIVE SCORE                    0 non-null      float64
 3   NEGATIVE SCORE                    0 non-null      float64
 4   POLARITY SCORE                    0 non-null      float64
 5   SUBJECTIVITY SCORE                0 non-null      float64
 6   AVG SENTENCE LENGTH               0 non-null      float64
 7   PERCENTAGE OF COMPLEX WORDS       0 non-null      float64
 8   FOG INDEX                         0 non-null      float64
 9   AVG NUMBER OF WORDS PER SENTENCE  0 non-null      float64
 10  COMPLEX WORD COUNT                0 non-null      float64
 11  WORD COUNT                        0 non-null      float64
 12  SYLLABLE

## **MERGING THE DATAFRAMES**

In [60]:
excel_file = excel_file.drop(['POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE',
       'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH',
       'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX',
       'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT',
       'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH'],axis =1)

In [67]:
df_new['URL_ID'] = df_new['URL_ID'].astype('int64')
merged_df = pd.merge(df_new, excel_file, on='URL_ID',how = 'left')
print(merged_df)

     URL_ID  POSITIVE SCORE  NEGATIVE SCORE  POLARITY SCORE  \
0        84              20              12        0.250000   
1        81              22              35       -0.228070   
2        59              17              14        0.096774   
3        52              26               0        1.000000   
4       139               8               3        0.454545   
..      ...             ...             ...             ...   
106     114              48              48        0.000000   
107     129               5               7       -0.166667   
108      91               3               1        0.500000   
109      74              34              25        0.152542   
110     133              26              20        0.130435   

     SUBJECTIVITY SCORE  AVG SENTENCE LENGTH  PERCENTAGE OF COMPLEX WORDS  \
0              0.012515            13.100000                    33.078880   
1              0.014372            14.441860                    24.315620   
2           

In [68]:
merged_df.to_csv('final.csv')

In [69]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111 entries, 0 to 110
Data columns (total 15 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   URL_ID                            111 non-null    int64  
 1   POSITIVE SCORE                    111 non-null    int64  
 2   NEGATIVE SCORE                    111 non-null    int64  
 3   POLARITY SCORE                    111 non-null    float64
 4   SUBJECTIVITY SCORE                111 non-null    float64
 5   AVG SENTENCE LENGTH               111 non-null    float64
 6   PERCENTAGE OF COMPLEX WORDS       111 non-null    float64
 7   FOG INDEX                         111 non-null    float64
 8   AVG NUMBER OF WORDS PER SENTENCE  111 non-null    float64
 9   COMPLEX WORD COUNT                111 non-null    int64  
 10  WORD COUNT                        111 non-null    int64  
 11  SYLLABLE PER WORD                 111 non-null    int64  
 12  PERSONAL